# Basic PCFG parser
This parser is implemented by basic PCFG without any preprocessing(ex. replace rare word). It is suggested to do any preprocess of data to get better performance Following are the file format

### input:
* 2 NONTERMINAL ADVP+ADJ
* 3 UNARYRULE NOUN British
* 164 BINARYRULE WHNP DET NP

the first indicates the nonterminal ADvp+ADJ was seen 2 times in training data<br>
the second indicates the UNARYRULE(X->Y) Noun->British was seen 3 times <br>
the final line indicate the BINARYRULE(X->Y+Z) when->DET+NP was seen 164 times<br>
in the training data

### sent:
json format:
["SBARQ", 
["WHNP+PRON", "What"], ["SBARQ", ["SQ", ["VERB", "is"], ["NP", ["ADJ", "blue"], ["NOUN", "power"]]], [".", "?"]]]

In [2]:
from __future__ import division
import json,sys
from collections import defaultdict

In [2]:
#The basic PCFG module
class PCFG:
    def __init__(self, nonterms,bin_rules,unary_rules,words):
        self.nonterms=dict(nonterms)
        self.bin_rules=dict(bin_rules)
        self.unary_rules=dict(unary_rules)
        self.words=words
        
        self.bin_rule_table={}
        
        for (a,b,c),count in bin_rules:
            self.bin_rule_table.setdefault(a,[])
            self.bin_rule_table[a].append((b,c))
    
    def nonterminals(self):
        return self.nonterms.keys()
    
    def has_unary(self,nonterm,word):
        return (nonterm,word) in self.unary_rules
    
    def rules(self,a):
        return [(a,b,c) for b,c in self.bin_rule_table.get(a,[])]
    
    def binary_rule_prob(self,r):
        return self.bin_rules[r]/self.nonterms[r[0]]
    
    def unary_rule_prob(self,r):
        return self.unary_rules[r]/self.nonterms[r[0]]
    
    
    @staticmethod  
    def read_data(handle):
        nonterms=[]
        bin_rules=[]
        unary_rules=[]
        words={}
        
        for l in handle:
            t=l.strip().split()
            #print t
            count=float(t[0])
            
            if t[1]=="NONTERMINAL":
                nonterms.append((t[2],count))
            if t[1]=="BINARYRULE":
                bin_rules.append(((t[2],t[3],t[4]),count))
            if t[1]=="UNARYRULE":
                unary_rules.append(((t[2],t[3]),count))
                words.setdefault(t[3],0)
                words[t[3]]+=count 
        return PCFG(nonterms,bin_rules,unary_rules,words)

In [7]:
def argmax(ls):
    if not ls:
        return None,0.0
    else:
        return max(ls,key=lambda x:x[1])

def backtrace(back,bp):
    if not back: return None
    if len(back)==6:
        (X,Y,Z,i,s,j)=back
        return [X,backtrace(bp[i,s,Y],bp),backtrace(bp[s+1,j,Z],bp)]
    else:
        (X,Y,i,i)=back
        return [X,Y]

In [41]:
#CKY algorithm
def CKY(pcfg,sentence):
    n=len(sentence)
    N=pcfg.nonterminals()
    x=[""]+sentence
    #print x
    def q1(X,Y):return pcfg.unary_rule_prob((X,Y))
    def q2(X,Y,Z):return pcfg.binary_rule_prob((X,Y,Z))
    
    pi=defaultdict(float)
    bp={}
    
    for i in range(1,n+1):
        for X in N:
            if pcfg.has_unary(X,x[i]):
                pi[i,i,X]=q1(X,x[i])
                bp[i,i,X]=(X,x[i],i,i)

    for l in range(1,n):
        for i in range(1,n-l+1):
            
            j=i+l
            for X in N: 
                back,score=argmax([((X,Y,Z,i,s,j),q2(X,Y,Z)*pi[i,s,Y]*pi[s+1,j,Z])\
                                   for s in range(i,j)\
                                   for X,Y,Z in pcfg.rules(X)
                                   if pi.get((i,s,Y),0.0)>0.0\
                                   if pi.get((s+1,j,Z),0.0)>0.0\
                                   ])
                if score>0.0:
                    bp[i,j,X],pi[i,j,X]=back,score
    # S is the special start symbol              
    if (1,n,"S") in pi:
        tree=backtrace(bp[1,n,"S"],bp)
        return tree   

In [1]:
def main(mode,count_file,sent_file):
    
    pcfg=PCFG.read_data(open(count_file))
    for i,l in enumerate(open(sent_file)):
        sentence=replace_rare_sentence(pcfg,l.strip().split())
        parse=CKY(pcfg,sentence)
        print json.dumps(parse)

if __name__=="__main__":
    main(sys.argv[1],sys.argv[2],sys.argv[3])

NameError: name 'sys' is not defined